In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t, anisotropic = []):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        self.anisotropic = anisotropic
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_anisotropic(self, energy_group_idx, energy_sub_group):
        return self.anisotropic[energy_group_idx][energy_sub_group]
        
        

In [3]:
energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [4]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [5]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [6]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [7]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [8]:
class Flux_estimator:
    def __init__(self, universe, group):
        self.boundaries = universe
        self.collision_sum = [0]
        self.group = group
        
    def add_collision(self, particle):
        if (self.boundaries.is_inside(particle) and particle.get_energy_group() == self.group):
            self.collision_sum[-1]  += particle.get_weight()
            

In [9]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [10]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [11]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [12]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [13]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [16]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [17]:
def calculate_mu(anisotropic_coeffs):
        
    return anisotropic_coeffs[1]/ anisotropic_coeffs[0]

In [127]:
def calculate_mu_qvadric(anisotropic_coeffs):
    
    b = 3. * anisotropic_coeffs[1] / anisotropic_coeffs[0]
    gamma =  random.uniform(0, 1)
    sqr = (1/2 - b) * (1/2 - b) + 2. * b * gamma
    

    result_mu1 = (-1/2 + math.sqrt(sqr))/b
    result_mu2 = (-1/2 - math.sqrt(sqr))/b
    if abs(result_mu1) < 1:
        return result_mu1
    
    if abs(result_mu2) < 1:
        return result_mu2

In [128]:
def calculate_mu_4(anisotropic_coeffs):
    
    a = (3/2 - 3/2 * (anisotropic_coeffs[1] / anisotropic_coeffs[0]))
    
    rand = random.uniform(0, 1)
    old_mu = 2. * math.sqrt(rand) - 1.
    f_mu = (1 + old_mu)/2.
    delta = 1
        
    mu = a * f_mu + (1 - a) * delta
        
    return mu 

In [129]:
def calculate_mu_dirac(anisotropic_coeffs):
    
    rand = random.uniform(0, 1)
    
    mu = 0
    if rand <= (1 - anisotropic_coeffs[1] / anisotropic_coeffs[0]):
        fi = random.uniform(0, 1)
        mu = 2. * fi - 1
    else:
        mu = 1
        
    return mu 

In [130]:
def get_anisotropic_direction_2(particle, mu):
    
    
    dir_x = particle.direction.tetta_x
    dir_y = particle.direction.tetta_y
    dir_z = particle.direction.tetta_z
    
    b = math.sqrt(1 - mu * mu)
    ro = math.sqrt(1 - dir_z * dir_z)
    phi = random.uniform(-math.pi, math.pi)

    c = math.cos(phi)
    d = math.sin(phi)
    a = mu
    
    dir_x_tmp = (b * c * dir_x * dir_z)/ro  - (b * d * dir_y)/ro + a * dir_x    
    dir_y_tmp = (b * c * dir_y * dir_z)/ro  + (b * d * dir_x)/ro + a * dir_y  
    dir_z_tmp = -b * c * ro  + a * dir_z
    
    particle.direction.tetta_x = dir_x_tmp
    particle.direction.tetta_y = dir_y_tmp
    particle.direction.tetta_z = dir_z_tmp

In [131]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx_new = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx_new)
    anisotropic_coeffs = c_s.get_anisotropic(energy_group_idx, energy_group_idx_new)
    mu = calculate_mu_4(anisotropic_coeffs)
  #  set_random_direction(particle)
    get_anisotropic_direction_2(particle, mu)


In [132]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    set_random_direction(particle)
    

In [133]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        process_scattering(particle, c_s)
        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [134]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [135]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    
    while not particle.is_terminated():

        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [136]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [137]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
     #   set_random_direction(particles[i])       

In [138]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [139]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [140]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [141]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [142]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [143]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [144]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [145]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [146]:
cs_fission_rr = [0.050632, 0.0010484]

cs_capture_rr = [0.025788, 0.0010046]

cs_scattering_rr = [[2.44383, 0.0], [0.029227, 0.62568]]

cs_total_rr = [2.52025, 0.65696]

cs_production_neutrons_rr = [2.50, 2.50]

cs_xi_table_rr = [ 0., 1.]
cs_anisotropic =[[[2.44383, 0.83318], [0., 0,]], [[0.029227, 0.0075737], [0.62568, 0.27459]]]


research_reactor_b = Sig(2, cs_fission_rr, cs_capture_rr, cs_scattering_rr, 
                       cs_production_neutrons_rr,  cs_xi_table_rr, cs_total_rr, cs_anisotropic)


energy_groups_2 = [0, 2 * 1E10]

In [147]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 9.4959)

y_m5 = Plane(0, 1, 0, -9.4959)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "black", "black", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -9.4959, -1000, 1000, 9.4959, 1000 )
slab.set_zero_point(0., 0., 0.)

In [148]:
estimators = []

In [149]:
cells = [slab]

In [150]:
materials = [research_reactor_b]

In [151]:
U235_Reactor = Universe(cells, materials)

In [152]:
test_number_of_particles = 10
test_number_interations = 100
test_number_inactive = 10

In [153]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, research_reactor_b, estimators)

 num1  5
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
 keff_cycle , k_effective_exp, std_k_effective 0.8150639064503491   0.8360299335079631  0.020966027057613912
i == 13
 keff_cycle , k_effective_exp, std_k_effective 0.7847299502678922   0.818929939094606  0.02095076543861119
i == 14
 keff_cycle , k_effective_exp, std_k_effective 0.8357729737718891   0.8231406977639268  0.015401226405813028
i == 15
 keff_cycle , k_effective_exp, std_k_effective 0.8570033865176403   0.8299132355146697  0.013718087784640481
i == 16
 keff_cycle , k_effective_exp, std_k_effective 0.8757568519876744   0.8375538382601704  0.01355861713545877
i == 17
 keff_cycle , k_effective_exp, std_k_effective 0.8021521872598486   0.8324964595458387  0.012525517094264622
i == 18
 keff_cycle , k_effective_exp, std_k_effective 0.7561782728965121   0.8229566862146729  0.014445542879001088
i == 19
 keff_cycle , k_effective_exp, std_k_effective 0.874261967182473   0.828657272988

 keff_cycle , k_effective_exp, std_k_effective 0.7955782016871469   0.8269017382158355  0.004177869540850242
i == 83
 keff_cycle , k_effective_exp, std_k_effective 0.7683259957083988   0.8260993307842268  0.004197647380926649
i == 84
 keff_cycle , k_effective_exp, std_k_effective 0.801099370615789   0.8257614934846533  0.004154293535289848
i == 85
 keff_cycle , k_effective_exp, std_k_effective 0.8586979781162764   0.8262006466130749  0.004121988935795943
i == 86
 keff_cycle , k_effective_exp, std_k_effective 0.786058760538816   0.8256724639015715  0.0041015416171954425
i == 87
 keff_cycle , k_effective_exp, std_k_effective 0.7715454350496859   0.8249695154749237  0.004108506826733828
i == 88
 keff_cycle , k_effective_exp, std_k_effective 0.860514125134913   0.8254252155987697  0.004081013947142709
i == 89
 keff_cycle , k_effective_exp, std_k_effective 0.7917719064604518   0.8249992243438543  0.0040514819570618725
i == 90
 keff_cycle , k_effective_exp, std_k_effective 0.8430887424249973

In [76]:
difference = (1. - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  14119.195727836375  with standart deviation [pcm]+- 417.9361494046775


In [77]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [78]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)